In [2]:
import os
import random as rn
from argparse import ArgumentParser
from os import mkdir
from os.path import exists
import sys
sys.path.insert(0, "../")

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau

2024-10-22 23:59:41.773229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 23:59:41.888672: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 23:59:41.924996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 23:59:42.132387: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-22 23:59:43.886127: W tensorflow/compiler/tf2

In [ ]:
seed = 0
os.environ["PYTHONHASHSEED"] = "0"

np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)

from deeparuco.impl.architectures import custom_regressor
from deeparuco.impl.datagen import corner_gen
from deeparuco.impl.losses import weighted_loss

# Control paramters
batch_size = 32
epochs = 100
patience = 10
reduce_after = 5

# Model
model = custom_regressor()
model.summary()
model.compile(loss="mae", optimizer="adam")

# Load dataset
train_src_dir = '../dataset/nested/crops/train'
valid_src_dir = '../dataset/nested/crops/val'
train_csv     = '../dataset/nested/crops/train128.csv'
valid_csv     = '../dataset/nested/crops/val128.csv'
train_df = pd.read_csv(train_csv)
valid_df = pd.read_csv(valid_csv)

train_generator = corner_gen(train_df, train_src_dir, batch_size, True, True)
valid_generator = corner_gen(valid_df, valid_src_dir, batch_size, True, True)

In [ ]:
run_name = 'reg_nested'

stop = EarlyStopping(
    monitor="val_loss",
    patience=patience,
    verbose=True,
    restore_best_weights=True,
    min_delta=1e-4,
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=reduce_after, factor=0.5)

# Training

if not exists("./models/nested/corner"):
    mkdir("./models/nested/corner")

if not exists("./models/nested/corner/losses"):
    mkdir("./models/nested/corner/losses")

csv_logger = CSVLogger(f"./models/nested/corner/losses/loss_{run_name}.csv")
model.fit(
    train_generator,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=[stop, reduce_lr, csv_logger],
    verbose=True,
)
model.save(f"./models/nested/corner/{run_name}.keras")
